First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
baseModel.fc = torch.nn.Linear(512, 512)
headModel = torch.nn.AdaptiveAvgPool2d((1,1))
hidden_layer = torch.nn.Linear(512, 128)
dropout_layer = torch.nn.Dropout(p=0.5)
output_layer = torch.nn.Linear(128, 2)
model = torch.nn.Sequential(baseModel, hidden_layer, torch.nn.ReLU(), output_layer)
model = model.to(device).eval()

Next, load the saved model.  Enter the model path you used to save.

In [2]:
model.load_state_dict(torch.load('model_resnet18_regression.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [4]:
torch.save(model_trt.state_dict(), 'steering_model_34_trt.pth')